<a href="https://colab.research.google.com/github/Oguzhandd/Team2_EnhancingPassengerSatisfaction/blob/main/google_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
services = ["Inflight wifi service", "Departure/Arrival time convenient", "Ease of Online booking",
            "Gate location","Food and drink", "Online boarding", "Seat comfort",
            "Inflight entertainment", "On-board service", "Leg room service",
            "Baggage handling", "Checkin service", "Inflight service",
            "Cleanliness"]

In [ ]:
import pandas as pd

# Load data
df1 = pd.read_csv("datasets/train.csv")
df2 = pd.read_csv("datasets/test.csv")
#
# print("Train set:\n")
# print(len(df1))
# print(df1.isnull().sum())
# print("\n")
# print("Test set:\n")
# print(len(df2))
# print(df2.isnull().sum())
# print("\n\n")


# Drop Null Raws
df1 = df1.dropna()
df2 = df2.dropna()


# return cleaned data of train set
def clean_data_train():
    return df1


# return cleaned data of test set
def clean_data_test():
    return df2


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import StratifiedKFold


def forward_feature_selection(classifier):
    sfK = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

    sfs = SFS(
        estimator=classifier,
        n_features_to_select='auto',
        tol=0.001,
        # chosen direction in the article
        direction='forward',
        # chosen scoring in the article
        scoring='roc_auc',
        # cv = 10 = 10-fold cross-validation
        cv=sfK,
    )
    return sfs


In [ ]:
from sklearn.preprocessing import LabelEncoder

train = clean_data_train()
test = clean_data_test()

# For Train set
lencoders = {}
for col in train.select_dtypes(include=['object']).columns:
    lencoders[col] = LabelEncoder()
    train[col] = lencoders[col].fit_transform(train[col])

# For Test Set
lencoders_t = {}
for col in test.select_dtypes(include=['object']).columns:
    lencoders_t[col] = LabelEncoder()
    test[col] = lencoders_t[col].fit_transform(test[col])
# Detect outliers
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1

# Removal of outliers from dataset
train = train[~((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR))).any(axis=1)]


def processed_data_train():
    return train


# return cleaned data of test set
def processed_data_test():
    return test


def split_to_train_test(train, test, features, target):
    train_cleaned = train
    test_cleaned = test

    features = features

    target = target  # Target variable

    X_train = train_cleaned[features]
    y_train = train_cleaned[target].to_numpy()  # Convert Series to NumPy array
    X_test = test_cleaned[features]
    y_test = test_cleaned[target].to_numpy()  # Convert Series to NumPy array

    return X_train, y_train, X_test, y_test


In [ ]:
from sklearn.ensemble import RandomForestClassifier

df_train_cleaned = clean_data_train()
df_test_cleaned = clean_data_test()

features = services
r_x_train, r_y_train, r_x_test, r_y_test = split_to_train_test(df_train_cleaned, df_test_cleaned, features, 'satisfaction')
random_forest = forward_feature_selection(RandomForestClassifier(n_estimators=10))
random_forest = random_forest.fit(r_x_train, r_y_train)
selected_features = random_forest.get_feature_names_out()

print(selected_features)

r_x_train, r_y_train, r_x_test, r_y_test = split_to_train_test(df_train_cleaned,
                                                       df_test_cleaned, selected_features, 'satisfaction')
random_forest = RandomForestClassifier(n_estimators=10)
random_forest.fit(r_x_train, r_y_train)
r_y_pred = random_forest.predict(x_train)


output: ['Inflight wifi service' 'Departure/Arrival time convenient'
 'Gate location' 'Online boarding' 'Inflight service']

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

df_train_cleaned = clean_data_train()
df_test_cleaned = clean_data_test()

k_x_train, k_y_train, k_x_test, k_y_test = split_to_train_test(df_train_cleaned, df_test_cleaned, services, 'satisfaction')
model = forward_feature_selection(KNeighborsClassifier(n_neighbors=10))
model = model.fit(k_x_train, k_y_train)
selected_features = model.get_feature_names_out()

print(selected_features)

k_x_train, k_y_train, k_x_test, k_y_test = split_to_train_test(df_train_cleaned,
                                                       df_test_cleaned, selected_features, 'satisfaction')
model = KNeighborsClassifier(n_neighbors=10)
model.fit(k_x_train, k_y_train)
k_y_pred = model.predict(k_x_train)

output: ['Inflight wifi service' 'Departure/Arrival time convenient'
 'Gate location' 'Online boarding' 'Inflight entertainment'
 'Leg room service' 'Inflight service']

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

df_train_cleaned = clean_data_train()
df_test_cleaned = clean_data_test()

features = services
g_x_train, g_y_train, g_x_test, g_y_test = split_to_train_test(df_train_cleaned, df_test_cleaned, features, 'satisfaction')
gb_model = forward_feature_selection(GradientBoostingClassifier(n_estimators=10))
gb_model = gb_model.fit(g_x_train, g_y_train)
selected_features = gb_model.get_feature_names_out()

print(selected_features)

g_x_train, g_y_train, g_x_test, g_y_test = split_to_train_test(df_train_cleaned,
                                                       df_test_cleaned, selected_features, 'satisfaction')
gb_model = GradientBoostingClassifier(n_estimators=10)
gb_model.fit(g_x_train, g_y_train)
g_y_pred = gb_model.predict(g_x_train)

output: ['Inflight wifi service' 'Departure/Arrival time convenient'
 'Online boarding' 'Leg room service']

In [ ]:
from sklearn.tree import DecisionTreeClassifier


df_train_cleaned = clean_data_train()
df_test_cleaned = clean_data_test()

d_x_train, d_y_train, d_x_test, d_y_test = split_to_train_test(df_train_cleaned, df_test_cleaned, services, 'satisfaction')

decision_tree = forward_feature_selection(DecisionTreeClassifier())
decision_tree = decision_tree.fit(d_x_train, d_y_train)
selected_features = decision_tree.get_feature_names_out()

print(selected_features)

d_x_train, d_y_train, d_x_test, d_y_test = split_to_train_test(df_train_cleaned,
                                                       df_test_cleaned, selected_features, 'satisfaction')
decision_tree = DecisionTreeClassifier()
decision_tree.fit(d_x_train, d_y_train)
d_y_pred = decision_tree.predict(d_x_train)

output: ['Inflight wifi service' 'Departure/Arrival time convenient'
 'Gate location' 'Online boarding' 'Inflight service']

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Random Forest Classifier
accuracy = accuracy_score(r_y_train, r_y_pred)
precision = precision_score(r_y_train, r_y_pred)
recall = recall_score(r_y_train, r_y_pred)
f1 = f1_score(r_y_train, r_y_pred)

print('Accuracy of random forest tree: ', accuracy)
print('Precision of random forest tree: ', precision)
print('Recall of random forest tree: ', recall)
print('F1 score of random forest tree: ', f1)

print('\n')
# K-Nearest Neighbor (kNN)
accuracy = accuracy_score(k_y_train, k_y_pred)
precision = precision_score(k_y_train, k_y_pred)
recall = recall_score(k_y_train, k_y_pred)
f1 = f1_score(k_y_train, k_y_pred)

print('Accuracy of random forest tree: ', accuracy)
print('Precision of random forest tree: ', precision)
print('Recall of random forest tree: ', recall)
print('F1 score of random forest tree: ', f1)

print('\n')
# Gradient Boosting Classifier
accuracy = accuracy_score(g_y_train, g_y_pred)
precision = precision_score(g_y_train, g_y_pred)
recall = recall_score(g_y_train, g_y_pred)
f1 = f1_score(g_y_train, g_y_pred)

print('Accuracy of random forest tree: ', accuracy)
print('Precision of random forest tree: ', precision)
print('Recall of random forest tree: ', recall)
print('F1 score of random forest tree: ', f1)

print('\n')
# Decision Tree
accuracy = accuracy_score(d_y_train, d_y_pred)
precision = precision_score(d_y_train, d_y_pred)
recall = recall_score(d_y_train, d_y_pred)
f1 = f1_score(d_y_train, d_y_pred)

print('Accuracy of random forest tree: ', accuracy)
print('Precision of random forest tree: ', precision)
print('Recall of random forest tree: ', recall)
print('F1 score of random forest tree: ', f1)


output:

Accuracy of random forest tree:  0.9236731857057359 |
Precision of random forest tree:  0.930998322147651 |
Recall of random forest tree:  0.8898367374212085 |
F1 score of random forest tree:  0.9099522827956132


Accuracy of random forest tree:  0.9311446608876962 |
Precision of random forest tree:  0.9435101235495843 |
Recall of random forest tree:  0.8946922956990445 |
F1 score of random forest tree:  0.9184529729853323


Accuracy of random forest tree:  0.8471533100372608 |
Precision of random forest tree:  0.8866303046428097 |
Recall of random forest tree:  0.742232220415618 |
F1 score of random forest tree:  0.8080308430930385 |


Accuracy of random forest tree:  0.9239048593547888 |
Precision of random forest tree:  0.9358690532265662 |
Recall of random forest tree:  0.88507027195581 |
F1 score of random forest tree:  0.9097610952757078